# Sudoku Solver using AI

Sudoku Solver using AI Techniques like **Constraint Propagation**, **Search** and **Elimination**

Here we discuss major techniques to solve a sudoku. 

1. **Naked Twins** 

    If two boxes within a unit have two possible values, they are said to be naked twins. This means that these _only_ these two boxes can have one of those two possible values, and hence these two values can be eliminated from the peers of these two boxes. This is the _Naked Twin_ strategy

2. **Diagonal Sudoku**

    In a diagonoal sudoku, the two big diagonals of the 9x9 square also have the additional constraint of uniqueness. The easiest way to solve a diagonoal sudoku is to add the diagonal boxes to the `unitlist` (that contains boxes that must have unique values).
    

Here are the key functions implemented:

**reduce_puzzle()** 
    Iteratively invoke  **eliminate()** and **only_choice()** to reduce the puzzle.
    
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    
**eliminate()**
    Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.
    
**only_choice()**
    Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.
    
**search()**

    Using depth-first search and propagation, create a search tree and solve the sudoku.
    Tries all possible values for a given box (using sub-trees)
    At any sub-tree, choose a box with the fewest possible values


In [1]:
## Sudoku solver

rows = 'ABCDEFGHI'
cols = '123456789'

def cross(a, b):
    '''  Return a char from a combined with char from b
    '''
    return [s + t for s in a for t in b]


In [2]:
# create Boxes - for each row/column
boxes = cross(rows, cols)

boxes

['A1',
 'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'E1',
 'E2',
 'E3',
 'E4',
 'E5',
 'E6',
 'E7',
 'E8',
 'E9',
 'F1',
 'F2',
 'F3',
 'F4',
 'F5',
 'F6',
 'F7',
 'F8',
 'F9',
 'G1',
 'G2',
 'G3',
 'G4',
 'G5',
 'G6',
 'G7',
 'G8',
 'G9',
 'H1',
 'H2',
 'H3',
 'H4',
 'H5',
 'H6',
 'H7',
 'H8',
 'H9',
 'I1',
 'I2',
 'I3',
 'I4',
 'I5',
 'I6',
 'I7',
 'I8',
 'I9']

In [3]:
## Now get the Row Units, Column Units and Square Units
# One UNIT == a set of 9 Boxes (row, column or a 9x9 box)

row_units = [cross(r, cols) for r in rows]
# rows_units[0] = ['A1', 'A2', ...'A9'] --> top most row

column_units = [cross(rows, c) for c in cols]
# column_units[0] = ['A1', 'B1', ... 'I1'] --> first column

square_units = [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]
# square_units[0] = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']


In [4]:
# Create the Units, and Peers
unitlist = row_units + column_units + square_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

In [5]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [6]:
## Grid values
# converts an input string '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
# into a dict:
# { 'A1': '.', 'A2': '.', 'A3': 3, 'A4': '.', 'A5': '2', ....}

inputs = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
len(inputs)

81

In [7]:
units

{'A1': [['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
  ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
  ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']],
 'A2': [['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
  ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
  ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']],
 'A3': [['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
  ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'],
  ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']],
 'A4': [['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
  ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4'],
  ['A4', 'A5', 'A6', 'B4', 'B5', 'B6', 'C4', 'C5', 'C6']],
 'A5': [['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
  ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5'],
  ['A4', 'A5', 'A6', 'B4', 'B5', 'B6', 'C4', 'C5', 'C6']],
 'A6': [['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
  ['A6', 'B6', 'C6', 'D6', 'E6'

In [8]:
peers

{'A1': {'A2',
  'A3',
  'A4',
  'A5',
  'A6',
  'A7',
  'A8',
  'A9',
  'B1',
  'B2',
  'B3',
  'C1',
  'C2',
  'C3',
  'D1',
  'E1',
  'F1',
  'G1',
  'H1',
  'I1'},
 'A2': {'A1',
  'A3',
  'A4',
  'A5',
  'A6',
  'A7',
  'A8',
  'A9',
  'B1',
  'B2',
  'B3',
  'C1',
  'C2',
  'C3',
  'D2',
  'E2',
  'F2',
  'G2',
  'H2',
  'I2'},
 'A3': {'A1',
  'A2',
  'A4',
  'A5',
  'A6',
  'A7',
  'A8',
  'A9',
  'B1',
  'B2',
  'B3',
  'C1',
  'C2',
  'C3',
  'D3',
  'E3',
  'F3',
  'G3',
  'H3',
  'I3'},
 'A4': {'A1',
  'A2',
  'A3',
  'A5',
  'A6',
  'A7',
  'A8',
  'A9',
  'B4',
  'B5',
  'B6',
  'C4',
  'C5',
  'C6',
  'D4',
  'E4',
  'F4',
  'G4',
  'H4',
  'I4'},
 'A5': {'A1',
  'A2',
  'A3',
  'A4',
  'A6',
  'A7',
  'A8',
  'A9',
  'B4',
  'B5',
  'B6',
  'C4',
  'C5',
  'C6',
  'D5',
  'E5',
  'F5',
  'G5',
  'H5',
  'I5'},
 'A6': {'A1',
  'A2',
  'A3',
  'A4',
  'A5',
  'A7',
  'A8',
  'A9',
  'B4',
  'B5',
  'B6',
  'C4',
  'C5',
  'C6',
  'D6',
  'E6',
  'F6',
  'G6',
  'H6',
  'I6'}

In [9]:
## 1st function - construct a Sudoku grid (dict) using given input string
def grid_values(grid):
    """ Convert a grid string defined by grid into a dict:
        {<box>: <value>} with a '.' for empties
        
        :param grid: sudoku grid in string
        
        :return a Sudoku grid dict, with
            - keys: Box labels, e.g. 'A1', 'B2', ...
            - values: Value in corresponding box, e.g. '9', '.', '1', etc 
    """
    assert len(grid) == 81, "Input must be a string 81 chars long (9x9)"
    return dict(zip(boxes, grid))

In [10]:
gv = grid_values(inputs)

In [11]:
gv

{'A1': '.',
 'A2': '.',
 'A3': '3',
 'A4': '.',
 'A5': '2',
 'A6': '.',
 'A7': '6',
 'A8': '.',
 'A9': '.',
 'B1': '9',
 'B2': '.',
 'B3': '.',
 'B4': '3',
 'B5': '.',
 'B6': '5',
 'B7': '.',
 'B8': '.',
 'B9': '1',
 'C1': '.',
 'C2': '.',
 'C3': '1',
 'C4': '8',
 'C5': '.',
 'C6': '6',
 'C7': '4',
 'C8': '.',
 'C9': '.',
 'D1': '.',
 'D2': '.',
 'D3': '8',
 'D4': '1',
 'D5': '.',
 'D6': '2',
 'D7': '9',
 'D8': '.',
 'D9': '.',
 'E1': '7',
 'E2': '.',
 'E3': '.',
 'E4': '.',
 'E5': '.',
 'E6': '.',
 'E7': '.',
 'E8': '.',
 'E9': '8',
 'F1': '.',
 'F2': '.',
 'F3': '6',
 'F4': '7',
 'F5': '.',
 'F6': '8',
 'F7': '2',
 'F8': '.',
 'F9': '.',
 'G1': '.',
 'G2': '.',
 'G3': '2',
 'G4': '6',
 'G5': '.',
 'G6': '9',
 'G7': '5',
 'G8': '.',
 'G9': '.',
 'H1': '8',
 'H2': '.',
 'H3': '.',
 'H4': '2',
 'H5': '.',
 'H6': '3',
 'H7': '.',
 'H8': '.',
 'H9': '9',
 'I1': '.',
 'I2': '.',
 'I3': '5',
 'I4': '.',
 'I5': '1',
 'I6': '.',
 'I7': '3',
 'I8': '.',
 'I9': '.'}

In [12]:
# Show the constructed Sudoku grid (dict)
display(gv)

. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


In [13]:
## 2nd func - Now return values for each box in grid
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    values = []
    all_digits = '123456789'
    for c in grid:
        if c == '.':  # i.e. blank 
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))

In [14]:
grid = grid_values(inputs)
display(grid)

123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

In [15]:
# Def an Eliminate func that eliminates values from peers that are not possible in a box

def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit, '')
    return values

In [16]:
## Test it out on the input string
grid = grid_values(inputs)
possible_values = eliminate(grid)
display(possible_values)

   45    4578    3   |   49     2     147  |   6     5789    57  
   9    24678    47  |   3      47     5   |   78    278     1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
  345    345     8   |   1     3456    2   |   9    34567  34567 
   7    123459   49  |  459   34569    4   |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  134    1347    2   |   6     478     9   |   5     1478    47  
   8     1467    47  |   2     457     3   |   17    1467    9   
   46    4679    5   |   4      1      47  |   3    24678   2467 


In [17]:
## Implement the Only Choice technique

def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

In [18]:
grid

{'A1': '45',
 'A2': '4578',
 'A3': '3',
 'A4': '49',
 'A5': '2',
 'A6': '147',
 'A7': '6',
 'A8': '5789',
 'A9': '57',
 'B1': '9',
 'B2': '24678',
 'B3': '47',
 'B4': '3',
 'B5': '47',
 'B6': '5',
 'B7': '78',
 'B8': '278',
 'B9': '1',
 'C1': '25',
 'C2': '257',
 'C3': '1',
 'C4': '8',
 'C5': '79',
 'C6': '6',
 'C7': '4',
 'C8': '23579',
 'C9': '2357',
 'D1': '345',
 'D2': '345',
 'D3': '8',
 'D4': '1',
 'D5': '3456',
 'D6': '2',
 'D7': '9',
 'D8': '34567',
 'D9': '34567',
 'E1': '7',
 'E2': '123459',
 'E3': '49',
 'E4': '459',
 'E5': '34569',
 'E6': '4',
 'E7': '1',
 'E8': '13456',
 'E9': '8',
 'F1': '1345',
 'F2': '13459',
 'F3': '6',
 'F4': '7',
 'F5': '3459',
 'F6': '8',
 'F7': '2',
 'F8': '1345',
 'F9': '345',
 'G1': '134',
 'G2': '1347',
 'G3': '2',
 'G4': '6',
 'G5': '478',
 'G6': '9',
 'G7': '5',
 'G8': '1478',
 'G9': '47',
 'H1': '8',
 'H2': '1467',
 'H3': '47',
 'H4': '2',
 'H5': '457',
 'H6': '3',
 'H7': '17',
 'H8': '1467',
 'H9': '9',
 'I1': '46',
 'I2': '4679',
 'I3': '5'

In [21]:
# test it out
only_choice_values = only_choice(possible_values)
display(only_choice_values)

  45    8     3   |  9     2     1   |  6    5789   57  
  9     6     47  |  3     4     5   |  8    278    1   
  2    257    1   |  8     7     6   |  4   23579  2357 
------------------+------------------+------------------
 345   345    8   |  1    3456   2   |  9   34567 34567 
  7     2     9   |  5   34569   4   |  1   13456   8   
 1345 13459   6   |  7    3459   8   |  2    1345  345  
------------------+------------------+------------------
 134   1347   2   |  6     8     9   |  5    1478   47  
  8    1467   47  |  2     5     3   |  17    6     9   
  6     9     5   |  4     1     7   |  3     8     2   


In [22]:
only_choice_values = only_choice(possible_values)
display(only_choice_values)

  4     8     3   |  9     2     1   |  6    5789   57  
  9     6     7   |  3     4     5   |  8     2     1   
  2     5     1   |  8     7     6   |  4     9     3   
------------------+------------------+------------------
 345   345    8   |  1    3456   2   |  9     7     6   
  7     2     9   |  5   34569   4   |  1   13456   8   
 1345 13459   6   |  7    3459   8   |  2    1345  345  
------------------+------------------+------------------
 134   1347   2   |  6     8     9   |  5     1     4   
  8    1467   4   |  2     5     3   |  7     6     9   
  6     9     5   |  4     1     7   |  3     8     2   


In [23]:
# Now reduce the puzzle by applying contraint propagation continuously

def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    
    Input: A sudoku in dictionary form.
    
    Output: The resulting sudoku in dictionary form.
    """
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        # Your code here: Use the Eliminate Strategy
        values = eliminate(values)

        # Your code here: Use the Only Choice Strategy
        values = only_choice(values)

        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [24]:
# test out the whole thing
grid = grid_values(inputs)
solved_values = reduce_puzzle(grid)
display(solved_values)

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


The Sudoku is solved! 👍👍


Let's try another grid. 

In [25]:
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values2 = grid_values(grid2)
solved2 = reduce_puzzle(values2)
display(solved2)

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


Clearly this is not solved. Let's see if there's another strategy to solve this Sudoku.

In [26]:
## Search method using Depth First Search

def search(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    
    if values is False:
        return False    ## fail early
    
    if all(len(values[b]) == 1 for b in boxes):
        return values   ## solved!
    
    # Choose one of the unfilled squares with the fewest possibilities
    n,b = min((len(values[b]), b) for b in boxes if len(values[b]) > 1)
    
    # Now use recursion to solve each one of the resulting sudokus, and if one returns a value (not False), return that answer!
    for value in values[b]:
        new_sudoku = values.copy()
        new_sudoku[b] = value
        
        # try search on a sub-tree:
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [27]:
## Try solving grid2 using Search()
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values2 = grid_values(grid2)

search2 = search(values2)
display(search2)

4 1 7 |3 6 9 |8 2 5 
6 3 2 |1 5 8 |9 4 7 
9 5 8 |7 2 4 |3 1 6 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 3 2 
3 4 6 |9 1 2 |7 5 8 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 7 3 |2 9 1 |6 8 4 
1 6 4 |8 7 5 |2 9 3 


We solved it!! 👍🍾

In [29]:
## 3rd func - Now return values for each box in grid
def grid_values(grid, show_dot=False):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
        show_dot: Show '.' if True, else show all digits ('123456789')
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    values = []
    all_digits = '123456789'
    for c in grid:
        if c == '.':  # i.e. blank
            if show_dot:
                values.append('.')
            else:
                values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))

In [30]:
## Solve function
def solve(input):
    ''' Solve a Sudoku puzzle, using a given input string.
        Displays the Before/After grids to show if the Sudoku is solved
        
        :param input - a string in format '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
        
        :return None
    '''
    assert len(input) == 81, "Input must be 81 chars (9x9) string representing the Sudoku"
    
    # 0 - Show the Before puzzle
    before_puzzle = grid_values(input, show_dot=True)
    print('BEFORE')
    display(before_puzzle)
    
    # 1 - Convert to grid
    grid = grid_values(input)
    
    # 2 - search (recursive) within the grid
    solved_values = search(grid)
    
    # 3 - show results
    print('\nAFTER')
    display(solved_values)
    
    return

In [31]:
#test it
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
solve(grid2)

BEFORE
4 . . |. . . |8 . 5 
. 3 . |. . . |. . . 
. . . |7 . . |. . . 
------+------+------
. 2 . |. . . |. 6 . 
. . . |. 8 . |4 . . 
. . . |. 1 . |. . . 
------+------+------
. . . |6 . 3 |. 7 . 
5 . . |2 . . |. . . 
1 . 4 |. . . |. . . 

AFTER
4 1 7 |3 6 9 |8 2 5 
6 3 2 |1 5 8 |9 4 7 
9 5 8 |7 2 4 |3 1 6 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 3 2 
3 4 6 |9 1 2 |7 5 8 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 7 3 |2 9 1 |6 8 4 
1 6 4 |8 7 5 |2 9 3 


In [32]:
# another test: 
grid3 = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
solve(grid3)

BEFORE
2 . . |. . . |. . . 
. . . |. . 6 |2 . . 
. . 1 |. . . |. 7 . 
------+------+------
. . 6 |. . 8 |. . . 
3 . . |. 9 . |. . 7 
. . . |6 . . |4 . . 
------+------+------
. 4 . |. . . |8 . . 
. . 5 |2 . . |. . . 
. . . |. . . |. . 3 

AFTER
2 3 9 |8 7 4 |1 5 6 
7 5 4 |3 1 6 |2 9 8 
6 8 1 |9 5 2 |3 7 4 
------+------+------
4 7 6 |1 2 8 |5 3 9 
3 1 2 |4 9 5 |6 8 7 
5 9 8 |6 3 7 |4 1 2 
------+------+------
1 4 3 |7 6 9 |8 2 5 
9 6 5 |2 8 3 |7 4 1 
8 2 7 |5 4 1 |9 6 3 
